In [48]:
from classe_maze import *

# <u>**SAE A Maze In Python**</u>

#### FRAILLON Mathis - MORLET Steven

<br>

### **Table des matières :**

1. **[Préambule](#1-préambule-1)**
2. **[Modélisation d'un labyrinthe](#2-modélisation-dun-labyrinthe)**
3. **[Implémentation](#3-implémentation)**
4. **[Manipulation de labyrinthes](#4-manipulation-de-labyrinthes)**
5. **[Génération](#5-génération)**
    1. [Arbre binaire](#51-arbre-binaire)
    2. [Sidewinder](#52-sidewinder)
    3. [Fusion de chemins](#53-fusion-de-chemins)
    4. [Exploration exhaustive](#54-exploration-exhaustive)
    5. [L'algorithme de Wilson](#55-lalgorithme-de-wilson)
6. **[Résolution](#6-résolution)**
    1. [Résolution par parcours](#61-résolution-par-parcours)
    2. [Résolution en aveugle : "la main droite"](#62-résolution-en-aveugle---la-main-droite)
7. **[Evaluation](#7-evaluation)**

<br>

<br>

# **1 Préambule**

[Table des matières](#table-des-matières)

SAE à rendre pour au plus tard le <u>**15 mars 2023 a 23h59**</u>.

<br>
<br>
<br>

# **2 Modélisation d’un labyrinthe**

[Table des matières](#table-des-matières)

<u>**Sommets du graphe :**</u>

(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3), (3, 0), (3, 1), (3, 2), (3, 3)

<u>**Arrêtes du graphe :**</u>

{(0, 0), (1, 0)}, {(1, 0), (2, 0)}, {(2, 0), (3, 0)}, {(2, 0), (2, 1)}, ...

<u>**Successeurs du graphes :**</u>

{ (0, 0): {(1, 0)}, (0, 1): {(0, 2), (1, 1)}, (0, 2): {(0, 1), (0, 3)}, (0, 3): {(0, 2), (1, 3)}, (1, 0): {(2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 2)}, (1, 2): {(1, 1), (2, 2)}, (1, 3): {(2, 3), (0, 3)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(2, 0), (2, 2)}, (2, 2): {(1, 2), (2, 1)}, (2, 3): {(3, 3), (1, 3)}, (3, 0): {(3, 1), (2, 0)}, (3, 1): {(3, 2), (3, 0)}, (3, 2): {(3, 1)}, (3, 3): {(2, 3)} }


<br>
<br>
<br>

# **3 Implémentation**

[Table des matières](#table-des-matières)

<br>

#### **La classe Maze par défaut**

<br>

<u>**Attributs de la classe :**</u>

**Height**, le nombre de lignes (int) de la grille du labyrinthe (autrement dit, la hauteur, en nombre de cellules).

**Width**, le nombre de colonnes (int) de la grille du labyrinthe (autrement dit, la hauteur, en nombre de cellules).

**Neighbors** : un dictionnaire (dict) qui associe à chaque cellule, un set contenant ses voisines (c’est-à-dire les cellules qu’on peut atteindre en un déplacement1, sans être bloqué par un mur).

<br>

<u>**Rappel du code :**</u>

    class Maze:
        """
        Classe Labyrinthe
        Représentation sous forme de graphe non-orienté
        dont chaque sommet est une cellule (un tuple (l,c))
        et dont la structure est représentée par un dictionnaire
        - clés : sommets
        - valeurs : ensemble des sommets voisins accessibles
        """
        def __init__(self, height, width):
            """
            Constructeur d'un labyrinthe de height cellules de haut 
            et de width cellules de large 
            Les voisinages sont initialisés à des ensembles vides
            Remarque : dans le labyrinthe créé, chaque cellule est complètement emmurée
            """
            self.height    = height
            self.width     = width
            self.neighbors = {(i,j): set() for i in range(height) for j in range (width)}

        def info(self):
            """
            **NE PAS MODIFIER CETTE MÉTHODE**
            Affichage des attributs d'un objet 'Maze' (fonction utile pour deboguer)
            Retour:
                chaîne (string): description textuelle des attributs de l'objet
            """
            txt = "**Informations sur le labyrinthe**\n"
            txt += f"- Dimensions de la grille : {self.height} x {self.width}\n"
            txt += "- Voisinages :\n"
            txt += str(self.neighbors)+"\n"
            valid = True
            for c1 in {(i, j) for i in range(self.height) for j in range(self.width)}:
                for c2 in self.neighbors[c1]:
                    if c1 not in self.neighbors[c2]:
                        valid = False
                        break
                else:
                    continue
                break
            txt += "- Structure cohérente\n" if valid else f"- Structure incohérente : {c1} X {c2}\n"
            return txt

        def __str__(self):
            """
            **NE PAS MODIFIER CETTE MÉTHODE**
            Représentation textuelle d'un objet Maze (en utilisant des caractères ascii)
            Retour:
                chaîne (str) : chaîne de caractères représentant le labyrinthe
            """
            txt = ""
            # Première ligne
            txt += "┏"
            for j in range(self.width-1):
                txt += "━━━┳"
            txt += "━━━┓\n"
            txt += "┃"
            for j in range(self.width-1):
                txt += "   ┃" if (0,j+1) not in self.neighbors[(0,j)] else "    "
            txt += "   ┃\n"
            # Lignes normales
            for i in range(self.height-1):
                txt += "┣"
                for j in range(self.width-1):
                    txt += "━━━╋" if (i+1,j) not in self.neighbors[(i,j)] else "   ╋"
                txt += "━━━┫\n" if (i+1,self.width-1) not in self.neighbors[(i,self.width-1)] else "   ┫\n"
                txt += "┃"
                for j in range(self.width):
                    txt += "   ┃" if (i+1,j+1) not in self.neighbors[(i+1,j)] else "    "
                txt += "\n"
            # Bas du tableau
            txt += "┗"
            for i in range(self.width-1):
                txt += "━━━┻"
            txt += "━━━┛\n"

            return txt

<br>
<br>

#### **Ajouter / Enlever un mur**


- **Ajouter** un mur entre une cellule c1 et une cellule c2 revient à **diminuer deux** voisinages, d’une cellule chacun ; il faut retirer c1 du voisinage de c2 **et** retirer c2 du voisinage de c1 .  

> ``laby.neighbors[(i, j)].remove((y,x))``

- **Casser** un mur entre une cellule c1 et une cellule c2 revient à **augmenter deux** voisinages ; il faut ajouter c1 au voisinage de c2 **et** ajouter c2 au voisinage de c1.  

> ``laby.neighbors[(i, j)].add((y,x))``

La méthode **info()** fournie teste la cohérence des voisinages en vérifiant que, dès lors qu’une cellule c1 est dans le voisinage d’une cellule c2, alors c2 est aussi dans le voisinage de c1.

<br>
<br>

#### **A Faire 1**

Modifier le constructeur par défaut en lui ajoutant l’argument **empty**, un booléen qui indique si le graphe doit être créé avec aucun mur, ou avec tous les murs.  

Modifier le corps de la méthode de telle manière que :  

- si **empty** vaut **True**, chaque cellule a pour voisines celles qui lui sont contigües dans la grille ;
- si **empty** vaut **False**, aucune cellule n’a de voisines.  

<br>

In [49]:
laby = Maze(5, 5, empty = True)
print(laby.neighbors)
print(laby)

{(0, 0): {(0, 1), (1, 0)}, (0, 1): {(1, 1), (0, 2), (0, 0)}, (0, 2): {(0, 1), (1, 2), (0, 3)}, (0, 3): {(1, 3), (0, 2), (0, 4)}, (0, 4): {(0, 3), (1, 4)}, (1, 0): {(1, 1), (2, 0), (0, 0)}, (1, 1): {(0, 1), (1, 0), (1, 2), (2, 1)}, (1, 2): {(1, 1), (0, 2), (1, 3), (2, 2)}, (1, 3): {(2, 3), (1, 2), (0, 3), (1, 4)}, (1, 4): {(1, 3), (2, 4), (0, 4)}, (2, 0): {(1, 0), (2, 1), (3, 0)}, (2, 1): {(3, 1), (1, 1), (2, 0), (2, 2)}, (2, 2): {(2, 3), (3, 2), (1, 2), (2, 1)}, (2, 3): {(2, 4), (3, 3), (1, 3), (2, 2)}, (2, 4): {(2, 3), (3, 4), (1, 4)}, (3, 0): {(3, 1), (4, 0), (2, 0)}, (3, 1): {(3, 2), (4, 1), (2, 1), (3, 0)}, (3, 2): {(3, 1), (3, 3), (4, 2), (2, 2)}, (3, 3): {(2, 3), (3, 2), (3, 4), (4, 3)}, (3, 4): {(4, 4), (2, 4), (3, 3)}, (4, 0): {(4, 1), (3, 0)}, (4, 1): {(3, 1), (4, 0), (4, 2)}, (4, 2): {(3, 2), (4, 1), (4, 3)}, (4, 3): {(4, 4), (3, 3), (4, 2)}, (4, 4): {(3, 4), (4, 3)}}
┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫

In [50]:
laby = Maze(5, 5, empty = False)
print(laby.neighbors)
print(laby)

{(0, 0): set(), (0, 1): set(), (0, 2): set(), (0, 3): set(), (0, 4): set(), (1, 0): set(), (1, 1): set(), (1, 2): set(), (1, 3): set(), (1, 4): set(), (2, 0): set(), (2, 1): set(), (2, 2): set(), (2, 3): set(), (2, 4): set(), (3, 0): set(), (3, 1): set(), (3, 2): set(), (3, 3): set(), (3, 4): set(), (4, 0): set(), (4, 1): set(), (4, 2): set(), (4, 3): set(), (4, 4): set()}
┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



<br>
<br>
<br>

# **4 Manipulation de labyrinthes**

[Table des matières](#table-des-matières)

<br>

<br>

#### **A Faire 2**

Écrire les méthodes d’instance suivantes :  

- **remove_wall(c1, c2)** qui supprime un mur entre deux cellules
- **get_walls()** qui retourne la liste de **tous les murs** sous la forme d’une liste de **tuple** de cellules
- **fill()** qui ajoute tous les murs possibles dans le labyrinthe
- **empty()** qui supprime tous les murs du labyrinthe  
( /!\ **Ici on renomme la méthode empty() en be_empty() à cause d'une erreur renvoyée par notre éditeur** /!\ )

- **get_contiguous_cells(c)** qui retourne la liste des cellules contigües à **c dans la grille** (sans s’occuper des éventuels murs)
- **get_reachable_cells(c)** qui retourne la liste des cellules accessibles depuis **c** (c’est-à-dire les cellules contiguës à c qui sont dans le voisinage de c)

<br>

<u> Tests d'utilisation de ces méthodes : </u>

In [51]:
laby = Maze(5, 5, empty = True)
laby.fill()
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [52]:
laby.remove_wall((0, 0), (0, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃       ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [53]:
laby.be_empty()
laby.add_wall((0, 0), (0, 1))
laby.add_wall((0, 1), (1, 1))
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃               ┃
┣   ╋━━━╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┣   ╋   ╋   ╋   ╋   ┫
┃                   ┃
┗━━━┻━━━┻━━━┻━━━┻━━━┛



In [54]:
print(laby.get_walls())

[[(0, 0), (0, 1)], [(0, 1), (1, 1)]]


In [55]:
print(laby.get_contiguous_cells((0,1)))

[(1, 1), (0, 0), (0, 2)]


In [56]:
print(laby.get_reachable_cells((0,1)))

[(0, 2)]


<br>
<br>
<br>

# **5 Génération**

[Table des matières](#table-des-matières)

<br>

## 5.1 Arbre binaire

<br>

<u>**Algorithme de construction par arbre binaire**</u>

```
- Initialisation : un labyrinthe plein (contenant tous les murs possibles)
 
- Pour chaque cellule du labyrinthe : 
 
    - Supprimer aléatoirement le mur EST ou le mur SUD (s’il n’en possède qu’un, supprimer ce mur ; s’il n’en possède aucun des deux, ne rien faire)
```

<br>

#### **A Faire 3**

Écrire une **méthode de classe gen_btree(h, w)** qui génère un labyrinthe à h lignes et w colonnes, en utilisant l’algorithme de construction par arbre binaire.  

> Pour définir une méthode de classe en **python**, il est nécessaire de faire précéder la définition de la méthode par le décorateur **@classmethod**.

<br>

In [57]:
laby = Maze.gen_btree(4,4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ┫
┃   ┃       ┃   ┃
┣   ╋━━━╋   ╋   ┫
┃       ┃       ┃
┣━━━╋   ╋━━━╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



<br>

## 5.2 Sidewinder

<br>

<u>**Algorithme Sidewinder**</u>
 
```
- Initialisation : création d’un labyrinthe plein

- Pour i allant de 0 à hauteur-2 :

    - Initialiser une variable séquence comme liste vide
    - Pour j allant de 0 à largeur-2 :
        - Ajouter la cellule (i, j) à la séquence
        - Tirer à pile ou face :
            - Si c’est pile : Casser le mur EST de la cellule (i, j)
            - Si c’est face :
                - Casser le mur SUD d’une des cellules (choisie au hasard) qui constituent le séquence qui vient d’être terminée.
                - Réinitialiser la séquence à une liste vide

    - Ajouter la dernière cellule à la séquence
    - Tirer une cellule au sort dans la séquence et casser son mur SUD

- Casser tous les murs EST de la dernière ligne

- Retourner le labyrinthe
```

<br>

#### **A Faire 4**

Écrire une **méthode de classe gen_sidewinder(h, w)** qui génère une labyrinthe à h lignes et w colonnes, en utilisant l’algorithme de construction par arbre binaire.

<br>

In [58]:
laby = Maze.gen_sidewinder(4, 4)
print(laby)

┏━━━┳━━━┳━━━┳━━━┓
┃   ┃   ┃       ┃
┣   ╋   ╋   ╋━━━┫
┃   ┃           ┃
┣   ╋━━━╋━━━╋   ┫
┃   ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ┫
┃               ┃
┗━━━┻━━━┻━━━┻━━━┛



<br>

## 5.3 Fusion de chemins

<br>

<u>**Algorithme << par fusion de chemins >>**</u>
 
```
- Initialisation :
    - on remplit le labyrinthe avec tous les murs possibles
    - on labélise les cellules de 1 à 
    - on extrait la liste de tous les murs et on les « mélange » (on les permute aléatoirement)

- Pour chaque mur de la liste :
    - Si les deux cellules séparées par le mur n’ont pas le même label :
        - casser le mur
        - affecter le label de l’une des deux cellules, à l’autre, et à toutes celles qui ont le même label que la deuxième
```

<br>

#### **A Faire 5**

Écrire une **méthode de classe gen_fusion(h,w)** qui génère un labyrinthe, à h lignes et w colonnes, parfait, avec l’algorithme de **fusion de chemin**.

<br>

In [59]:
laby = Maze.gen_fusion(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃   ┃       ┃   ┃           ┃   ┃       ┃   ┃       ┃   ┃   ┃
┣   ╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋   ┫
┃       ┃       ┃   ┃   ┃   ┃       ┃           ┃   ┃       ┃
┣━━━╋━━━╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ┫
┃               ┃   ┃                   ┃       ┃   ┃   ┃   ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━┫
┃   ┃   ┃   ┃           ┃   ┃       ┃       ┃               ┃
┣━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ┫
┃   ┃   ┃   ┃   ┃   ┃       ┃   ┃           ┃   ┃   ┃       ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋━━━┫
┃           ┃       ┃   ┃   ┃       ┃   ┃       ┃   ┃       ┃
┣━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ┫
┃   ┃   ┃   ┃       ┃   ┃           ┃   ┃   ┃       ┃       ┃
┣   ╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃           ┃   ┃   ┃           ┃   ┃           ┃       ┃   ┃
┣   ╋   

<br>

## 5.4 Exploration exhaustive

<br>

<u>**Algorithme de génération par exploration**</u>
 
```
- Initialisation :
    - Choisir une cellule au hasard
    - Marquer cette cellule comme étant visitée
    - Mettre cette cellule dans sur une pile
- Tant que la pile n’est pas vide :
    - Prendre la cellule en haut de la pile et l’en retirer
    - Si cette cellule a des voisins qui n’ont pas encore été visités :
        - La remettre sur la pile
        - Choisir au hasard l’une de ses cellules contigües qui n’a pas été visitée
        - Casser le mur entre la cellule (celle qui a été dépilée) et celle qui vient d’être choisie
        - Marquer la cellule qui vient d’être choisie comme visitée
        - Et la mettre sur la pile
```


<br>

#### **A Faire 6**

Écrire une **méthode de classe gen_exploration(h,w)** qui génère un labyrinthe, à h lignes et w colonnes, parfait, avec l’algorithme d’**exploration exhaustive**.

<br>

In [60]:
laby = Maze.gen_exploration(15,15)
print(laby)

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃               ┃                       ┃       ┃           ┃
┣━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ┫
┃           ┃   ┃       ┃       ┃           ┃               ┃
┣   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃   ┃           ┃   ┃       ┃               ┃           ┃   ┃
┣   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃       ┃           ┃   ┃   ┃                   ┃   ┃   ┃
┣   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃       ┃                       ┃   ┃           ┃       ┃   ┃
┣━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋   ┫
┃       ┃                   ┃           ┃   ┃       ┃       ┃
┣   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━┫
┃   ┃   ┃   ┃   ┃               ┃                           ┃
┣━━━╋   ╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ┫
┃       ┃       ┃       ┃   ┃               ┃       ┃       ┃
┣   ╋━━━

<br>

## 5.5 L'algorithme de Wilson

<br>

<u>**Algorithme de Wilson**</u>
 
```
- Choisir une cellule au hasard sur la grille et la marquer
- Tant qu’il reste des cellules non marquées :
    - Choisir une cellule de départ au hasard, parmi les cellules non marquées
    - Effectuer une marche aléatoire jusqu’à ce qu’une cellule marquée soit atteinte (en cas de boucle, si la tête du snake se 
      mord la queue, « couper » la boucle formée [autrement dit, supprimer toutes étapes depuis le précédent passage])
    - Marquer chaque cellule du chemin, et casser tous les murs rencontrés, jusqu’à la cellule marquée
```


<br>

#### **A Faire 7**

Écrire une méthode de classe **gen_wilson** qui implémente cet algorithme.

<br>

In [61]:
#laby = Maze.gen_wilson(12, 12)
#print(laby)

<br>
<br>
<br>

# **6 Résolution**

[Table des matières](#table-des-matières)

<br>

## 6.1 Résolution par parcours

<br>

<u>**Algorithme de résolution par parcours (pour aller de la cellule D à la cellule A)**</u>
 
 <br>

**Parcours du graphe jusqu’à ce qu’on trouve A**  

```
- Initialisation :
    - Placer D dans la struture d’attente (file ou pile) et marquer D
    - Mémoriser l’élément prédécesseur de D comme étant D
- Tant qu’il reste des cellules non-marquées :
    - Prendre la « première » cellule et la retirer de la structure (appelons c, cette cellule)
    - Si c correspond à A :
        - C’est terminé, on a trouvé un chemin vers la cellule de destination
    - Sinon :
        - Pour chaque voisine de c :
            - Si elle n’est pas marquée :
                - La marquer
                - La mettre dans la structure d’attente
                - Mémoriser son prédécesseur comme étant c
```

**Reconstruction du chemin à partir des prédécesseurs**

```
- Initialiser c à A
- Tant que c n’est pas D :
    - ajouter c au chemin
    - mettre le prédécesseur de c dans c
- Ajouter D au chemin

- Retourner le chemin
```

<br>

#### **A Faire 8**

Implémentez cet algorithme dans une méthode d’instance **solve_dfs(start, stop)** qui prend la cellule de départ et la cellule d’arrivée comme arguments. Créer ensuite une méthode **solve_bfs(start, stop)** qui implémente cette fois un parcours en largeur.

<br>

In [62]:
laby = Maze.gen_fusion(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

solution = laby.solve_bfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *   *   * ┃ *   * ┃       ┃       ┃           ┃   ┃   ┃
┣   ╋━━━╋   ╋   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ┫
┃   ┃       ┃ *   * ┃ *   *   *   * ┃               ┃   ┃   ┃
┣━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋   ┫
┃                           ┃   ┃ *     ┃       ┃   ┃       ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ╋━━━┫
┃       ┃           ┃           ┃ * ┃ *   *   * ┃   ┃   ┃   ┃
┣   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋   ╋   ┫
┃   ┃           ┃   ┃   ┃   ┃   ┃ *   * ┃     *         ┃   ┃
┣━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃       ┃           ┃       ┃               ┃ *     ┃       ┃
┣   ╋━━━╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋   ╋━━━╋   ╋   ┫
┃       ┃   ┃                   ┃   ┃     *   *     ┃   ┃   ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ┫
┃   ┃       ┃           ┃   ┃       ┃     *     ┃       ┃   ┃
┣━━━╋   

In [63]:
laby = Maze.gen_exploration(15, 15)
solution = laby.solve_dfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

solution = laby.solve_bfs((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))

┏━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┳━━━┓
┃ D   *     ┃ *   *   *   *   *   *   *   *   *   *   * ┃   ┃
┣   ╋   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋━━━╋━━━╋━━━╋━━━╋   ╋   ┫
┃   ┃ *   *   * ┃           ┃   ┃           ┃ *   *   * ┃   ┃
┣   ╋━━━╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋   ╋   ╋━━━╋   ╋━━━╋━━━╋   ┫
┃           ┃       ┃               ┃       ┃ * ┃ *   *   * ┃
┣━━━╋   ╋━━━╋   ╋   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋   ╋━━━╋   ┫
┃       ┃       ┃   ┃               ┃   ┃   ┃ * ┃ * ┃   ┃ * ┃
┣   ╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋   ╋   ┫
┃       ┃           ┃   ┃       ┃       ┃     * ┃ * ┃ *   * ┃
┣━━━╋━━━╋━━━╋━━━╋   ╋   ╋   ╋━━━╋━━━╋   ╋━━━╋   ╋   ╋   ╋━━━┫
┃                   ┃   ┃           ┃       ┃ *   * ┃ *   * ┃
┣   ╋━━━╋━━━╋━━━╋━━━╋   ╋   ╋━━━╋   ╋━━━╋   ╋━━━╋━━━╋━━━╋   ┫
┃   ┃       ┃           ┃   ┃   ┃       ┃   ┃   ┃ *   * ┃ * ┃
┣   ╋   ╋   ╋   ╋━━━╋━━━╋   ╋   ╋━━━╋   ╋   ╋   ╋   ╋   ╋   ┫
┃   ┃   ┃   ┃       ┃   ┃   ┃       ┃   ┃   ┃ *   * ┃ *   * ┃
┣   ╋   

<br>

## 6.2 Résolution en aveugle : "la main droite"

<br>

In [64]:
"""
laby = Maze.gen_fusion(15, 15)
solution = laby.solve_rhr((0, 0), (14, 14))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(14, 14)] = 'A'
print(laby.overlay(str_solution))
"""

"\nlaby = Maze.gen_fusion(15, 15)\nsolution = laby.solve_rhr((0, 0), (14, 14))\nstr_solution = {c:'*' for c in solution}\nstr_solution[( 0,  0)] = 'D'\nstr_solution[(14, 14)] = 'A'\nprint(laby.overlay(str_solution))\n"

<br>
<br>
<br>

# **7 Evaluation**

[Table des matières](#table-des-matières)

<br>

#### **A Faire 9**

Écrire une méthode d’instance **distance_geo(c1, c2)** qui calcule la distance « géodésique » entre la cellule **c1** et la cellule **c2** (vous pourrez utiliser une des méthodes de résolution implémentées avant), c’est à dire le nombre minimal de déplacements nécessaires sur le graphe pour aller de **c1** à **c2**.

Écrire une méthode d’instance **distance_man(c1, c2)** qui calcule la distance de Manhattan, **sur la grille**, entre la cellule **c1** et la cellule **c2**, c’est à dire le nombre minimal de déplacements nécessaires pour aller de **c2** à **c1** si le labyrinthe était vide de tout mur.

<br>

In [65]:
laby = Maze.gen_fusion(15, 15)
""" 
#Simple visualisation personnelle des chemins

print(laby.distance_geo((0,0),(7,7)))
solution = laby.solve_bfs((0, 0), (7, 7))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(7, 7)] = 'A'
print(laby.overlay(str_solution))
"""

" \n#Simple visualisation personnelle des chemins\n\nprint(laby.distance_geo((0,0),(7,7)))\nsolution = laby.solve_bfs((0, 0), (7, 7))\nstr_solution = {c:'*' for c in solution}\nstr_solution[( 0,  0)] = 'D'\nstr_solution[(7, 7)] = 'A'\nprint(laby.overlay(str_solution))\n"

In [66]:
laby = Maze.gen_fusion(15, 15)
""" 
#Simple visualisation personnelle des chemins

print(laby.distance_man((0,0),(7,7)))
laby.be_empty()
solution = laby.solve_bfs((0, 0), (7, 7))
str_solution = {c:'*' for c in solution}
str_solution[( 0,  0)] = 'D'
str_solution[(7, 7)] = 'A'
print(laby.overlay(str_solution))
"""

" \n#Simple visualisation personnelle des chemins\n\nprint(laby.distance_man((0,0),(7,7)))\nlaby.be_empty()\nsolution = laby.solve_bfs((0, 0), (7, 7))\nstr_solution = {c:'*' for c in solution}\nstr_solution[( 0,  0)] = 'D'\nstr_solution[(7, 7)] = 'A'\nprint(laby.overlay(str_solution))\n"

<br>
<br>
<br>

# **8 Partie BONUS non traitée**

[Table des matières](#table-des-matières)

<br>

#### <u>**Fin du notebook jupyter**</u>